In [19]:
# 1. Module Import
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [20]:
# 2. Data
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [21]:
# 3. Tokenizing
t = Tokenizer()
t.fit_on_texts([text])
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [22]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [23]:
# 4. Word To Indexing
sequences = list()
for line in text.split('\n'): # Wn을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [24]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [25]:
max_len=max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


In [26]:
# 5. Padding
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [27]:
# 6. Separate Array
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.

In [28]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [29]:
print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


In [30]:
# 7. One-Hot Encoding
y = to_categorical(y, num_classes=vocab_size)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [31]:
# 8. Module Import
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [32]:
# 9. Model Designing
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) # 레이블을 분리하였으므로 이제 X의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
# 10. Training
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.4905 - accuracy: 0.0000e+00
Epoch 2/200
1/1 - 0s - loss: 2.4756 - accuracy: 0.0000e+00
Epoch 3/200
1/1 - 0s - loss: 2.4610 - accuracy: 0.0909
Epoch 4/200
1/1 - 0s - loss: 2.4465 - accuracy: 0.1818
Epoch 5/200
1/1 - 0s - loss: 2.4319 - accuracy: 0.2727
Epoch 6/200
1/1 - 0s - loss: 2.4173 - accuracy: 0.3636
Epoch 7/200
1/1 - 0s - loss: 2.4024 - accuracy: 0.3636
Epoch 8/200
1/1 - 0s - loss: 2.3872 - accuracy: 0.4545
Epoch 9/200
1/1 - 0s - loss: 2.3716 - accuracy: 0.4545
Epoch 10/200
1/1 - 0s - loss: 2.3554 - accuracy: 0.4545
Epoch 11/200
1/1 - 0s - loss: 2.3386 - accuracy: 0.4545
Epoch 12/200
1/1 - 0s - loss: 2.3210 - accuracy: 0.4545
Epoch 13/200
1/1 - 0s - loss: 2.3027 - accuracy: 0.4545
Epoch 14/200
1/1 - 0s - loss: 2.2836 - accuracy: 0.4545
Epoch 15/200
1/1 - 0s - loss: 2.2636 - accuracy: 0.4545
Epoch 16/200
1/1 - 0s - loss: 2.2426 - accuracy: 0.4545
Epoch 17/200
1/1 - 0s - loss: 2.2207 - accuracy: 0.4545
Epoch 18/200
1/1 - 0s - loss: 2.1978 - accuracy: 

Epoch 147/200
1/1 - 0s - loss: 0.2084 - accuracy: 1.0000
Epoch 148/200
1/1 - 0s - loss: 0.2041 - accuracy: 1.0000
Epoch 149/200
1/1 - 0s - loss: 0.2000 - accuracy: 1.0000
Epoch 150/200
1/1 - 0s - loss: 0.1959 - accuracy: 1.0000
Epoch 151/200
1/1 - 0s - loss: 0.1920 - accuracy: 1.0000
Epoch 152/200
1/1 - 0s - loss: 0.1881 - accuracy: 1.0000
Epoch 153/200
1/1 - 0s - loss: 0.1843 - accuracy: 1.0000
Epoch 154/200
1/1 - 0s - loss: 0.1806 - accuracy: 1.0000
Epoch 155/200
1/1 - 0s - loss: 0.1770 - accuracy: 1.0000
Epoch 156/200
1/1 - 0s - loss: 0.1735 - accuracy: 1.0000
Epoch 157/200
1/1 - 0s - loss: 0.1700 - accuracy: 1.0000
Epoch 158/200
1/1 - 0s - loss: 0.1667 - accuracy: 1.0000
Epoch 159/200
1/1 - 0s - loss: 0.1634 - accuracy: 1.0000
Epoch 160/200
1/1 - 0s - loss: 0.1602 - accuracy: 1.0000
Epoch 161/200
1/1 - 0s - loss: 0.1571 - accuracy: 1.0000
Epoch 162/200
1/1 - 0s - loss: 0.1540 - accuracy: 1.0000
Epoch 163/200
1/1 - 0s - loss: 0.1510 - accuracy: 1.0000
Epoch 164/200
1/1 - 0s - loss: 

In [38]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
        print(model.predict(encoded,verbose=0))
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        print(result)
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [41]:
print(sentence_generation(model, t, '경마장에', 4))
# '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측

[[1.6776217e-03 9.5816687e-02 1.1128432e-03 8.3263773e-01 5.2639050e-03
  2.3145683e-03 1.5304046e-03 3.8671635e-02 1.9357376e-03 1.1611207e-02
  7.1987524e-03 2.2884166e-04]]
[3]
[[3.6078918e-04 9.5081723e-01 2.4928604e-04 3.3218469e-02 4.5372136e-03
  2.2177934e-03 4.6825974e-04 8.8215998e-04 5.9260306e-04 7.5200514e-04
  5.8872788e-03 1.6864133e-05]]
[1]
[[4.2479287e-04 7.7649797e-03 7.7332353e-04 5.1763714e-03 9.4197112e-01
  8.2378406e-03 7.5073552e-04 3.7080204e-04 1.1892428e-03 2.2834763e-02
  7.2150002e-04 9.7845346e-03]]
[4]
[[1.8969675e-03 6.3780812e-04 2.0372735e-03 5.3574692e-04 7.0819869e-03
  9.6888226e-01 1.0413317e-03 1.6089991e-05 2.5658773e-03 1.3924972e-03
  1.2278143e-02 1.6339039e-03]]
[5]
경마장에 있는 말이 뛰고 있다
